In [1]:
## Import necessary packages
import pandas as pd
import numpy as np
import tkinter as tk
import PyPDF2 
import tabula
import tkinter
import camelot
import ghostscript
import re
import os
import csv
import urllib
import io
import warnings
warnings.filterwarnings("ignore")

# Utah

In [1291]:
## -----------------------------------------------------------------
## (44) UTAH
print("Parsing Utah, 44/50...")
# get the full table
ut_pdfs = camelot.read_pdf("https://documents.deq.utah.gov/drinking-water/construction/DDW-2017-003850.pdf", pages='23-24', flavor = 'stream', row_tol=5, flag_size=True, split_text=True)
len(ut_pdfs)

Parsing Utah, 44/50...


2

In [1292]:
#select first page
ut = ut_pdfs[0].df
#drop title and column name rows
ut = ut.iloc[3:,].copy().reset_index(drop=True)

#create empty columns for splitting the combined column
ut['Priority.Points'] = ''
ut['PWSID'] = ''
ut['System.Name'] = ''

#for each row where Sys.Name/Borrower didn't split correctly, resplit and store the values
for i in ut.index:
    split_list = ut[1].iloc[i].split(" ", 3)
    ut['Priority.Points'].iloc[i] = split_list[0]
    ut['PWSID'].iloc[i] = "UT" + split_list[2]
    ut['System.Name'].iloc[i] = split_list[3]

#rename all columns
ut.rename(columns={0:'Status', 2:"Addl.Subsidy", 3:"Disadvantaged", 4:"Green.Project", 5:'Green.Amount', 6:"Equivalency.Project",
                   7:"County", 8:"Pop", 9:"Project.Title", 10:"Project.Total", 11:"SRF.Assistance", 12:"Principal.Forgiveness",
                   13:"ARPA.Funds.Authorized", 14:"BIL.Funds.Authorized", 15:"Funds.Authorized", 16:"Project.Segments.Sour", 17:"Project.Segments.Treat",
                   18:"Project.Segments.Stor", 19:"Project.Segments.Dist"}, inplace=True)
#remove linebreaks
ut = ut.replace("\n", "", regex=True)

ut.drop(columns=[1], inplace=True)

#subset to final columns
ut = ut[['Priority.Points', 'PWSID', 'System.Name', 'Status', 'Addl.Subsidy', 'Disadvantaged', 'Green.Project',
       'Green.Amount', 'Equivalency.Project', 'County', 'Pop', 'Project.Title',
       'Project.Total', 'SRF.Assistance', 'Principal.Forgiveness',
       'ARPA.Funds.Authorized', 'BIL.Funds.Authorized', 'Funds.Authorized',
       'Project.Segments.Sour', 'Project.Segments.Treat',
       'Project.Segments.Stor', 'Project.Segments.Dist']]


In [1293]:
#generally repeat the above process
ut1 = ut_pdfs[1].df
ut1 = ut1.iloc[3:12,].copy().reset_index(drop=True)

ut1['Priority.Points'] = ''
ut1['PWSID'] = ''
ut1['System.Name'] = ''

# for each row where Sys.Name/Borrower didn't split correctly, resplit and store the values
for i in ut1.index:
    split_list = ut1[1].iloc[i].split(" ", 3)
    ut1['Priority.Points'].iloc[i] = split_list[0]
    ut1['PWSID'].iloc[i] = "UT" + split_list[2]
    ut1['System.Name'].iloc[i] = split_list[3]

# manually fix splitting issue where priority points weren't present
ut1['Priority.Points'].iloc[7] = ""
ut1['Priority.Points'].iloc[8] = ""
ut1['PWSID'].iloc[7] = "UT19001"
ut1['PWSID'].iloc[8] = "UT19000"
ut1['System.Name'].iloc[7] = "Blanding City - Westwater #2"
ut1['System.Name'].iloc[8] = "San Juan Co. - NTUA Westwater #2"

ut1.rename(columns={0:'Status', 2:"Addl.Subsidy", 3:"Disadvantaged", 4:"Green.Project", 5:'Green.Amount', 6:"Equivalency.Project",
                   7:"County", 8:"Pop", 9:"Project.Title", 10:"Project.Total", 11:"SRF.Assistance", 12:"Funds.Authorized",
                   13:"Project.Segments.Sour", 14:"Project.Segments.Treat", 15:"Project.Segments.Stor", 16:"Project.Segments.Dist"}, inplace=True)

ut1 = ut1.replace("\n", "", regex=True)

ut1.drop(columns=[1], inplace=True)

# create missing columns 
ut1['Principal.Forgiveness'] = ''
ut1['ARPA.Funds.Authorized'] = ''
ut1['BIL.Funds.Authorized'] = ''

# correct merged cell and fill in sole missing column value
ut1['SRF.Assistance'].iloc[0] = '$1,609,000'
ut1['Principal.Forgiveness'].iloc[0] = '$805,000'

ut1 = ut1[['Priority.Points', 'PWSID', 'System.Name', 'Status', 'Addl.Subsidy', 'Disadvantaged', 'Green.Project',
       'Green.Amount', 'Equivalency.Project', 'County', 'Pop', 'Project.Title',
       'Project.Total', 'SRF.Assistance', 'Principal.Forgiveness',
       'ARPA.Funds.Authorized', 'BIL.Funds.Authorized', 'Funds.Authorized',
       'Project.Segments.Sour', 'Project.Segments.Treat',
       'Project.Segments.Stor', 'Project.Segments.Dist']]

ut1

,Priority.Points,PWSID,System.Name,Status,Addl.Subsidy,Disadvantaged,Green.Project,Green.Amount,Equivalency.Project,County,...,Project.Total,SRF.Assistance,Principal.Forgiveness,ARPA.Funds.Authorized,BIL.Funds.Authorized,Funds.Authorized,Project.Segments.Sour,Project.Segments.Treat,Project.Segments.Stor,Project.Segments.Dist
0,10.4,UT09007,Panguitch,A,Y,Y,N,N,N,Garfield,...,"$1,629,000","$1,609,000","$805,000",,,,,,,10
1,10.4,UT07002,Myton Town,A,N,Y,N,N,N,Duchesne,...,"$650,000","$650,000",,,,,,,,
2,9.8,UT14063,Holden Town,A,N,N,N,N,N,Millard,...,"$6,721,000","$6,571,000",,,,,20,5,15,20
3,7.9,UT16011,Little Meadow Estates,A,N,###,N,N,N,Piute,...,"$246,000","$246,000",,,,"$ 246,000",,,,
4,7.5,UT11004,Enoch City,A,N,N,N,N,N,Iron,...,"$1,639,440","$645,000",,,,"$ 645,000",,,,
5,6.5,UT17001,Bridgerland Water Co.,A,N,N,N,N,N,Rich,...,"$1,128,170","$1,350,000",,,,"$ 1,350,000",10,,,
6,0.0,UT22005,Henefer (Secondary Irrigation),A,N,N,N,N,N,Summit,...,"$2,100,000","$2,100,000",,,,,,,,
7,,UT19001,Blanding City - Westwater #2,A,N,N,N,N,N,San Juan,...,"$5,683,073","$340,984",,,,"$ 340,984",,,,
8,,UT19000,San Juan Co. - NTUA Westwater #2,A,N,N,N,N,N,San Juan,...,"$4,355,105","$457,000",,,,"$ 457,000",,,,


In [1294]:
ut_final = pd.concat([ut, ut1])

ut_final.to_csv("final_ppls/44-Utah_Comprehensive.csv", index=False)